<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EB%82%A8%EC%83%81%EC%9A%B0_LSTM%EC%9D%84_%ED%99%9C%EC%9A%A9%ED%95%9C_%EC%83%9D%EC%84%B1AI_%EB%AA%A8%EB%8D%B8%EB%A7%81_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install kiwipiepy

In [ ]:
!pip install konlpy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset

In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/project_final/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# 데이터 불러오기

### 전체 데이터셋

In [ ]:
yo_review =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi.csv")['고객리뷰']
yo_reply = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi.csv")['사장댓글']
yo_star = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi.csv")['별점']

train = pd.concat([yo_review, yo_reply, yo_star], axis=1)

rename_cols = ['review', 'reply', 'star']
train.columns = rename_cols

# shuffled_df = train.sample(frac=1, random_state=2)  # frac=1은 모든 행을 선택, random_state는 재현 가능한 랜덤 설정
# train = shuffled_df.iloc[:1000].reset_index(drop=True)

### 오버샘플링

In [ ]:
from sklearn.utils import resample

# 오버샘플링을 수행할 별점 분포 계산
star_distribution = train['star'].value_counts().sort_index()

print(star_distribution)

1       92
2       64
3      179
4      560
5    17791
Name: star, dtype: int64


In [ ]:
# 오버샘플링을 수행할 최대 샘플 수 설정
max_samples = 2000

# 오버샘플링 수행
oversampled_dfs = []

for star, count in star_distribution.items():
    subset = train[train['star'] == star]
    # 별점이 5가 아닌 경우에만 오버샘플링 수행
    if star != 5:
        resampled = resample(subset, replace=True, n_samples=max_samples, random_state=SEED)
        oversampled_dfs.append(resampled)
    else:
        shuffled_dfs = train[train['star'] == 5].sample(frac=1, random_state=2).reset_index(drop=True)
        subset = shuffled_dfs.iloc[:2000] # 별점이 5인 경우는 2000개 랜덤 추출해서 전체 비율 맞추기
        oversampled_dfs.append(subset)

In [ ]:
# 오버샘플링된 데이터프레임 합치기
train = pd.concat(oversampled_dfs).reset_index(drop=True)

In [ ]:
# 오버샘플링 후 별점 분포 확인
ov_dis = train['star'].value_counts().sort_index()
ov_dis

1    2000
2    2000
3    2000
4    2000
5    2000
Name: star, dtype: int64

In [ ]:
# 별점 컬럼 삭제
train = train.drop(columns=['star'])
train.head()

,review,reply
0,처음배달온 사진..음식을 위아래로 흔들어왔나보네요\r\n씬도우가 도우만 씬이 아니고...,안녕하세요\n먼저 배달의 불편을 드려서 정말 죄송합니다\n대행기사님들께 다시한번 부...
1,머리카락나옴. 위생신경좀\r\n국수와 숙주 비율 1:1\r\n맛 없음. 이맛도 저맛...,안녕하세요 고객님ㅜㅜ\n다시 한 번 사과의 말씀드리고 싶습니다 ㅠㅠ 오늘 직원들과의...
2,별로에요......,저희 가성비를 자랑하는 초밥입니다~찾아주셔서감사합니다~
3,별점 1개도 아까워요,받으신 피자에 문제가 있으셨다면 매장으로 바로 전화 주시면 신속하게 해결 해드리겠습니다
4,오늘도 맛있게 먹게 해주셔서 감사합니다.\n그런데 지난번 누룽지 사건 이후로 사장님...,안녕하세요 고객님\n저희 매장 다시 찾아주셨는데 만족 드리지 못해 죄송합니다\n고의...


## 데이터 정제

- 불필요한 문자 삭제

In [ ]:
review_list = []
for review in train['review']:
    if isinstance(review, str):
        # pat = re.compile("[^a-zA-Z가-힣ㄱ-ㅎ0-9]")
        # pat = re.compile("[^a-zA-Z가-힣ㄱ-ㅎ0-9\(\)\{\}\[\]]")
        # pat = re.compile("[^a-zA-Z가-힣0-9\(\)\{\}\[\]]")
        pat = re.compile("[^a-zA-Z가-힣0-9]")
        # pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", review)

        # result = re.sub(r'\.{2,}', '.', result)
        # result = re.sub(r'\,{2,}', ',', result)
        # result = re.sub(r'\~{2,}', '~', result)
        # result = re.sub(r'\!{2,}', '!', result)
        # result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['review'] = review_list
train.head()

,review,reply
0,처음배달온 사진 음식을 위아래로 흔들어왔나보네요 씬도우가 도우만 씬이 아니고 모든 ...,안녕하세요\n먼저 배달의 불편을 드려서 정말 죄송합니다\n대행기사님들께 다시한번 부...
1,머리카락나옴 위생신경좀 국수와 숙주 비율 1 1 맛 없음 이맛도 저맛도 아닌맛 배고...,안녕하세요 고객님ㅜㅜ\n다시 한 번 사과의 말씀드리고 싶습니다 ㅠㅠ 오늘 직원들과의...
2,별로에요,저희 가성비를 자랑하는 초밥입니다~찾아주셔서감사합니다~
3,별점 1개도 아까워요,받으신 피자에 문제가 있으셨다면 매장으로 바로 전화 주시면 신속하게 해결 해드리겠습니다
4,오늘도 맛있게 먹게 해주셔서 감사합니다 그런데 지난번 누룽지 사건 이후로 사장님 심...,안녕하세요 고객님\n저희 매장 다시 찾아주셨는데 만족 드리지 못해 죄송합니다\n고의...


In [ ]:
reply_list = []
for reply in train['reply']:
    if isinstance(reply, str):
        # pat = re.compile("[^a-zA-Z가-힣ㄱ-ㅎ0-9]")
        # pat = re.compile("[^a-zA-Z가-힣ㄱ-ㅎ0-9\(\)\{\}\[\]]")
        # pat = re.compile("[^a-zA-Z가-힣0-9\(\)\{\}\[\]]")
        pat = re.compile("[^a-zA-Z가-힣0-9]")
        # pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", reply)

        # result = re.sub(r'\.{2,}', '.', result)
        # result = re.sub(r'\,{2,}', ',', result)
        # result = re.sub(r'\~{2,}', '~', result)
        # result = re.sub(r'\!{2,}', '!', result)
        # result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['reply'] = reply_list
train.head()

,review,reply
0,처음배달온 사진 음식을 위아래로 흔들어왔나보네요 씬도우가 도우만 씬이 아니고 모든 ...,안녕하세요 먼저 배달의 불편을 드려서 정말 죄송합니다 대행기사님들께 다시한번 부탁드...
1,머리카락나옴 위생신경좀 국수와 숙주 비율 1 1 맛 없음 이맛도 저맛도 아닌맛 배고...,안녕하세요 고객님 다시 한 번 사과의 말씀드리고 싶습니다 오늘 직원들과의 미팅을 통...
2,별로에요,저희 가성비를 자랑하는 초밥입니다 찾아주셔서감사합니다
3,별점 1개도 아까워요,받으신 피자에 문제가 있으셨다면 매장으로 바로 전화 주시면 신속하게 해결 해드리겠습니다
4,오늘도 맛있게 먹게 해주셔서 감사합니다 그런데 지난번 누룽지 사건 이후로 사장님 심...,안녕하세요 고객님 저희 매장 다시 찾아주셨는데 만족 드리지 못해 죄송합니다 고의적으...


### "~~님" → "고객님"으로 변환

In [ ]:
train['reply'] = train['reply'].str.replace(r'\w+\s*님', '고객님', regex=True)

### 가게명 키워드 컬럼화(나중에 필요할 때 사용)

In [ ]:
# ygy_store_lst = ["1988응답하라추억의옛날도시락-방이점", "워커스커피 로스터스", "떡깨비-가락점", "당치땡-헬리오시티점", "우시장국밥-석촌점",  "고돼지-송파점",  "마라의신마라탕&마라샹궈",  "춘리마라탕-송파점",  "손수경의육회담은연어-가락점",  "무국적식탁-송파점",  "청년치킨-삼전점",  "송파만다린",  "배달의쌀국수-송파점",  "닭강점-헬리오시티점",  "토핑폭탄김치찜&김치찌개-송파점",  "조형훈족발보쌈",  "여왕의브런치",  "모어댄버터",  "플러스82-송파점",  "더티베지",  "잭아저씨족발보쌈-본점",  "육회&연어-미아본점",  "갈비민족-본점",  "구구족-성신여대역점",  "피자보이시나-대학로점",  "왕빈자삼파전-미아본점",  "행복한찜닭-성북점",  "스시사소우",  "저팔계&중국&마라탕&덮밥&요리&전문점",  "명작파스타-성북점",  "올데이파스타-동대문구점",  "대한냉면-성북점",  "인생제육",  "고기듬뿍미트박스-성북점",  "피자스웨그-성북점",  "청춘식당묵은지김치찜",  "삽교원조두리곱창",  "뜸들이다-북서울꿈의숲점",  "더팔당매운오징어&닭발 김치찜",  "우리집반찬도시락-혜화막내딸지점",  "수피자",  "마라쿡",  "김포불닭발",  "헬로팬닭갈비&마약볶음밥",  "대찌 대파생고기김치찌개&정육왕국물두루치기",  "구래상회",  "승도리네곱도리탕",  "자성당",  "수락",  "나인곱창",  "어시장",  "돈까스회관",  "호랑이초밥",  "땡초곱창막창",  "피자이탈리",  "텐텐마라탕",  "과일에반하다",  "삼다수",  "병천토속순대",  "최고남제육명가",  "짱닭치킨",  "걸작떡볶이",  "직화삼겹직구삼",  "짜글이가돼었소",  "부대찌개자신있는집",  "토핑폭탄김치찜&김치찌개-부천점",  "1989마라탕-부천점",  "미친피자-부평점",  "천년닭강정-부천부평점",  "수상한삼겹살-부천점",  "수제죽전문점-본가진죽-본점",  "오늘은 분식",  "이끌림마라탕",  "탐나는피자-부평점",  "고기혁명-부천점",  "라화방마라탕-신중동점",  "전금례닭볶음탕-부천점",  "큰아들백순대",  "공주닭발-부천시청점",  "신자매김치찜&김치찌개",  "짱닭치킨-도촌점",  "이태리면가게-야탑점",  "폴트버거-판교점",  "랜돌프뉴욕페페로니-야탑점",  "홍싸롱-수제돈까스&파스타",  "쇼부덮밥",  "카산도-정자본점",  "부성초밥-미금본점",  "닭장수섭삼계탕&전기구이통닭",  "깨돌이김밥-미금점",  "울진죽변항",  "GTS버거-분당정자점",  "코브라독스-분당미금역점",  "마라하오-죽전점",  "땅끝수산",  "초밥대통령-용인수지점",  "올데이케밥&샐러드",  "야키토리 카마쿠라",  "비오키친-서현본점",  "부산오빠1인국밥-분당점",  "알촌",  "밈피자",  "고씨네카레",  "홍스족발",  "뜸들이다",  "감성커피",  "짚신스시",  "떡형",  "39도시락",  "피제이피자",  "황궁쟁반짜장",  "OK반점",  "모모타코야키&카페",  "꼬알라파이",  "진지덮밥",  "피자프라텔로",  "한식세끼1인김치찜&김치찌개",  "윤희횟집",  "시민보쌈족발&감자탕",  "카페봄봄",  "킹프레소빅와플",  "최가네한쌈",  "창타이누들",  "보돌미역",  "홍대개미"  ]

# store_name = []
# # region_name = []

# for store in ygy_store_lst:
#     pat = re.split(r'[- &]', store)
#     pat = ''.join(pat)
#     store_name.append(pat)
#     # if len(pat) > 1 and pat[1] and not pat[1].isdigit():
#     #     region_name.append(pat[1])

In [ ]:
# store_name[:8]

In [ ]:
# short_name = []
# for name in store_name:
#     for length in [2, 3, 4]:
#         cut = name[:length]
#         short_name.append(cut)
#     if name[0].isdigit(): # 숫자로 시작하면 텍스트가 나오는 부분까지 추출
#         cut = name[:6]
#         short_name.append(cut)

# short_name = [name for name in short_name if not name.isdigit()] # 숫자로만 되어 있는 키워드 삭제
# short_name = list(set(short_name))
# print(len(short_name), short_name)

In [ ]:
# # short_name 에서 불용어 처리
# drop_name = ['OK','갈비','감성','걸작','고기','공주','과일','김포','대한','땡초','명작','미친','배달','병천','부대','부산','비오','삽교','손수','송파','쇼부','수상','수제','시민','오늘', '오늘은', '우리', '울진', '이끌', '이끌림', '인생', '직화', '진지', '천년', '청년', '청춘', '최고', '큰아', '큰아들', '탐나', '토핑', '행복', '행복한', '헬로', '홍대',]
# short_name = [col for col in short_name if col not in drop_name]
# print(len(short_name),short_name)

In [ ]:
# train['store'] = ""

# # 각 가게명에서 가능한 단축된 형태를 확인하고 store 컬럼에 추가
# for name in short_name:
#     train[name] = train['reply'].apply(lambda x: name if name in x else '')
#     train['store'] = train.apply(lambda row: row['store'] + ',' + row[name] if row[name] else row['store'], axis=1)

# # 첫 번째 문자가 콤마인 경우 제거
# train['store'] = train['store'].apply(lambda x: x[1:] if x.startswith(',') else x)

# # 불필요한 컬럼 삭제
# train = train.drop(columns=short_name)

# train.head()

In [ ]:
# train[train['reply'].str.contains(short_name[2])].index

In [ ]:
# train['store'][10216]

- 정제된 데이터 저장하기

In [ ]:
# # 전체
# # train.to_csv(f"{DATA_PATH}yogiyo_clean.csv", index=False)

# # 오버샘플
# train.to_csv(f"{DATA_PATH}yogiyo_OV_clean.csv", index=False)

### 정제된 데이터 불러오기

In [ ]:
# 전체
# train = pd.read_csv(f"{DATA_PATH}yogiyo_clean.csv")

# 오버샘플
train = pd.read_csv(f"{DATA_PATH}yogiyo_OV_clean.csv")

review, reply = train["review"] , train["reply"]
len(review) , len(reply)

(10000, 10000)

# 한국어 형태소 분석

In [ ]:
review[:5]

0    처음배달온 사진 음식을 위아래로 흔들어왔나보네요 씬도우가 도우만 씬이 아니고 모든 ...
1    머리카락나옴 위생신경좀 국수와 숙주 비율 1 1 맛 없음 이맛도 저맛도 아닌맛 배고...
2                                                별로에요 
3                                          별점 1개도 아까워요
4    오늘도 맛있게 먹게 해주셔서 감사합니다 그런데 지난번 누룽지 사건 이후로 사장님 심...
Name: review, dtype: object

In [ ]:
import unicodedata

def normalize(sentence):
    return unicodedata.normalize('NFC', sentence)

normalize(review[10])

'초밥 횟감이 신선하지 않았습니다 저녁 챙겨드시라고 선물해 드린 분께도 죄송한 마음입니다 '

In [ ]:
from konlpy.tag import Mecab, Okt

def clean_text(sentence):
    okt = Okt()
    sentence = normalize(sentence)
    sentence = okt.morphs(sentence)
    sentence = ' '.join(sentence)
    sentence = sentence.lower()
    return sentence

clean_text(review[0])

'처음 배달 온 사진 음식 을 위아래 로 흔들어 왔나 보네요 씬도우 가 도우 만 씬 이 아니고 모든 재료 가 씬입니 다 치즈 맛 은 찾아볼 래야 찾아볼수도 없고 돈 아까워서 환불 하려다 귀찮아서 그냥 먹는데 이집 은 오늘이 마지막 여태 먹은 피자 중 최악 이네 요'

# 단어 사전 만들기

In [ ]:
PAD_TOKEN = 0
SOS_TOKEN = 1
EOS_TOKEN = 2

class WordVocab():
    def __init__(self):
        self.word2index = {'<PAD>': PAD_TOKEN,
                           '<SOS>': SOS_TOKEN,
                           '<EOS>': EOS_TOKEN,}
        self.word2count = {}
        self.index2word = {PAD_TOKEN: '<PAD>',
                           SOS_TOKEN: '<SOS>',
                           EOS_TOKEN: '<EOS>'}
        self.n_words = 3  # PAD, SOS, EOS 포함

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
print(f'원문: {review[0]}')
lang = WordVocab()
lang.add_sentence(review[0])
print('단어사전: ', lang.word2index)

원문: 처음배달온 사진 음식을 위아래로 흔들어왔나보네요 씬도우가 도우만 씬이 아니고 모든 재료가 씬입니다 치즈맛은 찾아볼래야 찾아볼수도 없고 돈아까워서 환불하려다 귀찮아서 그냥 먹는데 이집은 오늘이 마지막 여태 먹은 피자 중 최악이네요 
단어사전:  {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '처음배달온': 3, '사진': 4, '음식을': 5, '위아래로': 6, '흔들어왔나보네요': 7, '씬도우가': 8, '도우만': 9, '씬이': 10, '아니고': 11, '모든': 12, '재료가': 13, '씬입니다': 14, '치즈맛은': 15, '찾아볼래야': 16, '찾아볼수도': 17, '없고': 18, '돈아까워서': 19, '환불하려다': 20, '귀찮아서': 21, '그냥': 22, '먹는데': 23, '이집은': 24, '오늘이': 25, '마지막': 26, '여태': 27, '먹은': 28, '피자': 29, '중': 30, '최악이네요': 31, '': 32}


# 데이터셋 만들기

In [ ]:
class TextDataset(Dataset):
    def __init__(self, df, min_length=3, max_length=32):
        super().__init__()

        # TOKEN 정의
        self.PAD_TOKEN = 0 # Padding 토큰
        self.SOS_TOKEN = 1 # SOS 토큰
        self.EOS_TOKEN = 2 # EOS 토큰

        self.tagger = Okt()   # 형태소 분석기
        self.max_length = max_length # 한 문장의 최대 길이 지정

        # src: 리뷰, tgt: 답변
        src_clean = []
        tgt_clean = []

        # 단어 사전 생성
        wordvocab = WordVocab()

        for _, row in df.iterrows():
            src = row['review']
            tgt = row['reply']

            # 한글 전처리
            src = self.clean_text(src)
            tgt = self.clean_text(tgt)

            if len(src.split()) > min_length and len(tgt.split()) > min_length:
                # 최소 길이를 넘어가는 문장의 단어만 추가
                wordvocab.add_sentence(src)
                wordvocab.add_sentence(tgt)
                src_clean.append(src)
                tgt_clean.append(tgt)

        self.srcs = src_clean
        self.tgts = tgt_clean
        self.wordvocab = wordvocab

    def clean_text(self, sentence):
        # 형태소 처리
        sentence = self.tagger.morphs(sentence)
        sentence = ' '.join(sentence)
        sentence = sentence.lower()
        return sentence

    def texts_to_sequences(self, sentence):
        # 문장 -> 시퀀스로 변환
        return [self.wordvocab.word2index[w] for w in sentence.split()]

    def pad_sequence(self, sentence_tokens):
        # 문장의 맨 끝 토큰은 제거
        sentence_tokens = sentence_tokens[:(self.max_length-1)]
        token_length = len(sentence_tokens)

        # 문장의 맨 끝부분에 <EOS> 토큰 추가
        sentence_tokens.append(self.EOS_TOKEN)

        for i in range(token_length, (self.max_length-1)):
            # 나머지 빈 곳에 <PAD> 토큰 추가
            sentence_tokens.append(self.PAD_TOKEN)
        return sentence_tokens

    def __getitem__(self, idx):
        inputs = self.srcs[idx]
        inputs_sequences = self.texts_to_sequences(inputs)
        inputs_padded = self.pad_sequence(inputs_sequences)

        outputs = self.tgts[idx]
        outputs_sequences = self.texts_to_sequences(outputs)
        outputs_padded = self.pad_sequence(outputs_sequences)

        return torch.tensor(inputs_padded), torch.tensor(outputs_padded)

    def __len__(self):
        return len(self.srcs)

In [ ]:
MAX_LENGTH = 27

dataset = TextDataset(train, min_length=3, max_length=MAX_LENGTH)

In [ ]:
x, y = dataset[0]

In [ ]:
print(f'x shape: {x.shape}')
print(x)

x shape: torch.Size([27])
tensor([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
        21, 22, 15, 23, 24, 25, 26, 27,  2])


In [ ]:
print(f'y shape: {y.shape}')
print(y)

y shape: torch.Size([27])
tensor([49, 50,  4, 51, 52,  8, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
        65, 66, 67, 68, 69, 70, 71, 72,  2])


## 데이터셋 분할

In [ ]:
# 80%의 데이터를 train에 할당합니다.
train_size = int(len(dataset) * 0.8)
train_size

7112

In [ ]:
# 나머지 20% 데이터를 test에 할당합니다.
test_size = len(dataset) - train_size
test_size

1778

In [ ]:
from torch.utils.data import random_split

# 랜덤 스플릿으로 분할을 완료합니다.
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# 데이터로더 만들기

In [ ]:
from torch.utils.data import DataLoader, SubsetRandomSampler

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
# 1개의 배치 데이터를 추출
x, y = next(iter(train_loader))

# shape: (batch_size, sequence_length)
x.shape, y.shape

(torch.Size([16, 27]), torch.Size([16, 27]))

# Encoder 만들기

In [ ]:
class Encoder(nn.Module):
    def __init__(self, num_vocabs, hidden_size, embedding_dim, num_layers):
        super().__init__()

        # 단어 사전의 개수 지정
        self.num_vocabs = num_vocabs
        # 임베딩 레이어 정의 (number of vocabs, embedding dimension)
        self.embedding = nn.Embedding(num_vocabs, embedding_dim)
        # GRU (embedding dimension)
        self.gru = nn.GRU(embedding_dim,
                          hidden_size,
                          num_layers=num_layers,
                          bidirectional=False)

    def forward(self, x):
        x = self.embedding(x).permute(1, 0, 2)
        output, hidden = self.gru(x)
        return output, hidden

In [ ]:
embedding_dim = 64 # 임베딩 차원
embedding = nn.Embedding(dataset.wordvocab.n_words, embedding_dim)

# x의 shape을 변경합니다.
# (batch_size, sequence_length) => (sequence_length, batch_size)
embedded = embedding(x)

print(x.shape)
print(embedded.shape)
# input:  (sequence_length, batch_size)
# output: (sequence_length, batch_size, embedding_dim)

torch.Size([16, 27])
torch.Size([16, 27, 64])


In [ ]:
embedded = embedded.permute(1, 0, 2)
print(embedded.shape)
# (sequence_length, batch_size, embedding_dim)

torch.Size([27, 16, 64])


In [ ]:
hidden_size = 32

gru = nn.GRU(embedding_dim,      # embedding 차원
             hidden_size,
             num_layers=1,
             bidirectional=False)

# input       : (sequence_length, batch_size, embedding_dim)
# h0          : (Bidirectional(1) x number of layers(1), batch_size, hidden_size)
o, h = gru(embedded, None)

print(o.shape)
print(h.shape)
# output      : (sequence_length, batch_size, hidden_size x bidirectional(1))
# hidden_state: (bidirectional(1) x number of layers(1), batch_size, hidden_size)

torch.Size([27, 16, 32])
torch.Size([1, 16, 32])


In [ ]:
NUM_VOCABS = dataset.wordvocab.n_words
print(f'number of vocabs: {NUM_VOCABS}')

number of vocabs: 11589


## Encoder 정의

In [ ]:
# Encoder 정의
encoder = Encoder(NUM_VOCABS, hidden_size=32, embedding_dim=64, num_layers=1)

In [ ]:
# Encoder에 x 통과 후 output, hidden_size 의 shape 확인
# input(x)    : (batch_size, sequence_length)
o, h = encoder(x)

print(o.shape)
print(h.shape)
# output      : (sequence_length, batch_size, hidden_size x bidirectional(1))
# hidden_state: (bidirectional(1) x number of layers(1), batch_size, hidden_size)

torch.Size([27, 16, 32])
torch.Size([1, 16, 32])


# Decoder 만들기

In [ ]:
class Decoder(nn.Module):
    def __init__(self, num_vocabs, hidden_size, embedding_dim, num_layers=1, dropout=0.2):
        super().__init__()
        # 단어사전 개수
        self.num_vocabs = num_vocabs
        self.embedding = nn.Embedding(num_vocabs, embedding_dim)
        self.dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(embedding_dim,
                          hidden_size,
                          num_layers=num_layers,
                          bidirectional=False)

        # 최종 출력은 단어사전의 개수
        self.fc = nn.Linear(hidden_size, num_vocabs)

    def forward(self, x, hidden_state):
        x = x.unsqueeze(0) # (1, batch_size) 로 변환
        embedded = F.relu(self.embedding(x))
        embedded = self.dropout(embedded)
        output, hidden = self.gru(embedded, hidden_state)
        output = self.fc(output.squeeze(0)) # (sequence_length, batch_size, hidden_size(32) x bidirectional(1))
        return output, hidden

In [ ]:
x = torch.abs(torch.randn(size=(1, 16)).long())
print(x)
x.shape
# batch_size = 16 이라 가정했을 때,
# (1, batch_size)
# 여기서 batch_size => (1, batch_size) 로 shape 변환을 선행

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])


torch.Size([1, 16])

In [ ]:
embedding_dim = 64 # 임베딩 차원
embedding = nn.Embedding(dataset.wordvocab.n_words, embedding_dim)

embedded = embedding(x)
embedded.shape
# embedding 출력
# (1, batch_size, embedding_dim)

torch.Size([1, 16, 64])

In [ ]:
hidden_size = 32

gru = nn.GRU(embedding_dim,
             hidden_size,
             num_layers=1,
             bidirectional=False,
             batch_first=False, # batch_first=False로 지정
            )

o, h = gru(embedded)

print(o.shape)
# output shape: (sequence_length, batch_size, hidden_size(32) x bidirectional(1))
print(h.shape)
# hidden_state shape: (Bidirectional(1) x number of layers(1), batch_size, hidden_size(32))

torch.Size([1, 16, 32])
torch.Size([1, 16, 32])


In [ ]:
fc = nn.Linear(32, NUM_VOCABS) # 출력은 단어사전의 개수로 가정

output = fc(o[0])

print(o[0].shape)
print(output.shape)
# input : (batch_size, output from GRU)
# output: (batch_size, output dimension)

torch.Size([16, 32])
torch.Size([16, 11589])


## Decoder 정의

In [ ]:
decoder = Decoder(num_vocabs=dataset.wordvocab.n_words, hidden_size=32, embedding_dim=64, num_layers=1)

In [ ]:
x, y = next(iter(train_loader))

o, h = encoder(x)

print(o.shape, h.shape)
# output      : (batch_size, sequence_length, hidden_size(32) x bidirectional(1))
# hidden_state: (Bidirectional(1) x number of layers(1), batch_size, hidden_size(32))

torch.Size([27, 16, 32]) torch.Size([1, 16, 32])


In [ ]:
x = torch.abs(torch.full(size=(16,), fill_value=SOS_TOKEN).long())
print(x)
x.shape
# batch_size = 16 이라 가정(16개의 SOS 토큰)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


torch.Size([16])

In [ ]:
decoder_output, decoder_hidden = decoder(x, h)
decoder_output.shape, decoder_hidden.shape
# (batch_size, num_vocabs), (1, batch_size, hidden_size)

(torch.Size([16, 11589]), torch.Size([1, 16, 32]))

# Seq2Seq 모델 만들기

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, inputs, outputs, teacher_forcing_ratio=0.5):
        # inputs : (batch_size, sequence_length)
        # outputs: (batch_size, sequence_length)

        batch_size, output_length = outputs.shape
        output_num_vocabs = self.decoder.num_vocabs

        # 리턴할 예측된 outputs를 저장할 임시 변수
        # (sequence_length, batch_size, num_vocabs)
        predicted_outputs = torch.zeros(output_length, batch_size, output_num_vocabs).to(self.device)

        # 인코더에 입력 데이터 주입, encoder_output은 버리고 hidden_state 만 살립니다.
        # 여기서 hidden_state가 디코더에 주입할 context vector 입니다.
        # (Bidirectional(1) x number of layers(1), batch_size, hidden_size)
        _, decoder_hidden = self.encoder(inputs)

        # (batch_size) shape의 SOS TOKEN으로 채워진 디코더 입력 생성
        decoder_input = torch.full((batch_size,), SOS_TOKEN, device=self.device)

        # 순회하면서 출력 단어를 생성합니다.
        # 0번째는 SOS TOKEN이 위치하므로, 1번째 인덱스부터 순회합니다.
        for t in range(0, output_length):
            # decoder_input : 디코더 입력 (batch_size) 형태의 SOS TOKEN로 채워진 입력
            # decoder_output: (batch_size, num_vocabs)
            # decoder_hidden: (Bidirectional(1) x number of layers(1), batch_size, hidden_size), context vector와 동일 shape
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)

            # t번째 단어에 디코더의 output 저장
            predicted_outputs[t] = decoder_output

            # teacher forcing 적용 여부 확률로 결정
            # teacher forcing 이란: 정답치를 다음 RNN Cell의 입력으로 넣어주는 경우. 수렴속도가 빠를 수 있으나, 불안정할 수 있음
            teacher_force = random.random() < teacher_forcing_ratio

            # top1 단어 토큰 예측
            top1 = decoder_output.argmax(1)

            # teacher forcing 인 경우 ground truth 값을, 그렇지 않은 경우, 예측 값을 다음 input으로 지정
            decoder_input = outputs[:, t] if teacher_force else top1

        return predicted_outputs.permute(1, 0, 2) # (batch_size, sequence_length, num_vocabs)로 변경

In [ ]:
# Encoder 정의
encoder = Encoder(num_vocabs=dataset.wordvocab.n_words,
                       hidden_size=32,
                       embedding_dim=64,
                       num_layers=1)
# Decoder 정의
decoder = Decoder(num_vocabs=dataset.wordvocab.n_words,
                       hidden_size=32,
                       embedding_dim=64,
                       num_layers=1)
# Seq2Seq 정의
seq2seq = Seq2Seq(encoder, decoder, 'cpu')

In [ ]:
x, y = next(iter(train_loader))
print(x.shape, y.shape)
# (batch_size, sequence_length), (batch_size, sequence_length)

torch.Size([16, 27]) torch.Size([16, 27])


In [ ]:
output = seq2seq(x, y)
print(output.shape)
# (batch_size, sequence_length, num_vocabs)

torch.Size([16, 27, 11589])


## 모델 확인

In [ ]:
NUM_VOCABS = dataset.wordvocab.n_words
HIDDEN_SIZE = 512
EMBEDDIMG_DIM = 256

print(f'num_vocabs: {NUM_VOCABS}\n======================')

# Encoder 정의
encoder = Encoder(num_vocabs=NUM_VOCABS,
                  hidden_size=HIDDEN_SIZE,
                  embedding_dim=EMBEDDIMG_DIM,
                  num_layers=1)
# Decoder 정의
decoder = Decoder(num_vocabs=NUM_VOCABS,
                  hidden_size=HIDDEN_SIZE,
                  embedding_dim=EMBEDDIMG_DIM,
                  num_layers=1)

# Seq2Seq 생성
# encoder, decoder를 device 모두 지정
model = Seq2Seq(encoder.to(device), decoder.to(device), device)
print(model)

num_vocabs: 11589
Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(11589, 256)
    (gru): GRU(256, 512)
  )
  (decoder): Decoder(
    (embedding): Embedding(11589, 256)
    (dropout): Dropout(p=0.2, inplace=False)
    (gru): GRU(256, 512)
    (fc): Linear(in_features=512, out_features=11589, bias=True)
  )
)


# 하이퍼파라미터 정의

In [ ]:
class EarlyStopping:
    def __init__(self, patience=3, delta=0.0, mode='min', verbose=True):
        """
        patience (int): loss or score가 개선된 후 기다리는 기간. default: 3
        delta  (float): 개선시 인정되는 최소 변화 수치. default: 0.0
        mode     (str): 개선시 최소/최대값 기준 선정('min' or 'max'). default: 'min'.
        verbose (bool): 메시지 출력. default: True
        """
        self.early_stop = False
        self.patience = patience
        self.verbose = verbose
        self.counter = 0

        self.best_score = np.Inf if mode == 'min' else 0
        self.mode = mode
        self.delta = delta


    def __call__(self, score):

        if self.best_score is None:
            self.best_score = score
            self.counter = 0
        elif self.mode == 'min':
            if score < (self.best_score - self.delta):
                self.counter = 0
                self.best_score = score
                if self.verbose:
                    print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
            else:
                self.counter += 1
                if self.verbose:
                    print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                          f'Best: {self.best_score:.5f}' \
                          f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')

        elif self.mode == 'max':
            if score > (self.best_score + self.delta):
                self.counter = 0
                self.best_score = score
                if self.verbose:
                    print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
            else:
                self.counter += 1
                if self.verbose:
                    print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                          f'Best: {self.best_score:.5f}' \
                          f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')


        if self.counter >= self.patience:
            if self.verbose:
                print(f'[EarlyStop Triggered] Best Score: {self.best_score:.5f}')
            # Early Stop
            self.early_stop = True
        else:
            # Continue
            self.early_stop = False

In [ ]:
LR = 1e-3
optimizer = optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss()
es = EarlyStopping(patience=5, delta=0.001, mode='min', verbose=True)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

# 학습 loop 만들기

In [ ]:
def train(model, data_loader, optimizer, loss_fn, device):
    model.train()
    running_loss = 0

    for x, y in data_loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()

        # output: (batch_size, sequence_length, num_vocabs)
        output = model(x, y)
        output_dim = output.size(2)

        # 1번 index 부터 슬라이싱한 이유는 0번 index가 SOS TOKEN 이기 때문
        # (batch_size*sequence_length, num_vocabs) 로 변경
        output = output.reshape(-1, output_dim)

        # (batch_size*sequence_length) 로 변경
        y = y.view(-1)

        # Loss 계산
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x.size(0)

    return running_loss / len(data_loader)

In [ ]:
def evaluate(model, data_loader, loss_fn, device):
    model.eval()

    eval_loss = 0

    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)
            output = model(x, y)
            output_dim = output.size(2)
            output = output.reshape(-1, output_dim)
            y = y.view(-1)

            # Loss 계산
            loss = loss_fn(output, y)

            eval_loss += loss.item() * x.size(0)

    return eval_loss / len(data_loader)

# 추론 함수

In [ ]:
def sequence_to_sentence(sequences, index2word):
    outputs = []
    for p in sequences:

        word = index2word[p]
        if p not in [SOS_TOKEN, EOS_TOKEN, PAD_TOKEN]:
            outputs.append(word)
        if word == EOS_TOKEN:
            break
    return ' '.join(outputs)

# 테스트 loop 만들기
- 챗봇 함수화

In [ ]:
def random_evaluation(model, dataset, index2word, device, n=10):

    n_samples = len(dataset)
    indices = list(range(n_samples))
    np.random.shuffle(indices)      # Shuffle
    sampled_indices = indices[:n]   # Sampling N indices

    # 샘플링한 데이터를 기반으로 DataLoader 생성
    sampler = SubsetRandomSampler(sampled_indices)
    sampled_dataloader = DataLoader(dataset, batch_size=10, sampler=sampler)

    model.eval()
    with torch.no_grad():
        for x, y in sampled_dataloader:
            x, y = x.to(device), y.to(device)
            output = model(x, y, teacher_forcing_ratio=0)
            # output: (number of samples, sequence_length, num_vocabs)

            preds = output.detach().cpu().numpy()
            x = x.detach().cpu().numpy()
            y = y.detach().cpu().numpy()

            for i in range(n):
                print(f'질문   : {sequence_to_sentence(x[i], index2word)}')
                print(f'답변   : {sequence_to_sentence(y[i], index2word)}')
                print(f'예측답변: {sequence_to_sentence(preds[i].argmax(1), index2word)}')
                print('==='*10)

# 학습하기

In [ ]:
# # yogiyo 전체데이터 모델링
# NUM_EPOCHS = 20
# STATEDICT_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_clean.pt'

# best_loss = np.inf

# for epoch in tqdm(range(NUM_EPOCHS), total=NUM_EPOCHS):
#     loss = train(model, train_loader, optimizer, loss_fn, device)

#     val_loss = evaluate(model, test_loader, loss_fn, device)

#     if val_loss < best_loss:
#         best_loss = val_loss
#         torch.save(model.state_dict(), STATEDICT_PATH)

#     if epoch % 5 == 0:
#         print(f'epoch: {epoch+1}, loss: {loss:.4f}, val_loss: {val_loss:.4f}')

#     # Early Stop
#     es(loss)
#     if es.early_stop:
#         break

#     # Scheduler
#     scheduler.step(val_loss)

# model.load_state_dict(torch.load(STATEDICT_PATH))
# torch.save(model.state_dict(), f'/content/drive/MyDrive/project_final/data/models/yogiyo_clean{best_loss:.4f}.pt')

  0%|          | 0/20 [00:00<?, ?it/s]

epoch: 1, loss: 80.9891, val_loss: 65.0345
[EarlyStopping] (Update) Best Score: 80.98909
[EarlyStopping] (Update) Best Score: 58.38377
[EarlyStopping] (Update) Best Score: 51.98271
[EarlyStopping] (Update) Best Score: 49.01400
[EarlyStopping] (Update) Best Score: 45.88895
epoch: 6, loss: 44.9918, val_loss: 51.0277
[EarlyStopping] (Update) Best Score: 44.99184
[EarlyStopping] (Update) Best Score: 42.37393
[EarlyStopping] (Update) Best Score: 41.04085
[EarlyStopping] (Update) Best Score: 39.98164
[EarlyStopping] (Update) Best Score: 38.58978
epoch: 11, loss: 37.4433, val_loss: 50.8560
[EarlyStopping] (Update) Best Score: 37.44329
Epoch 00011: reducing learning rate of group 0 to 5.0000e-04.
[EarlyStopping] (Update) Best Score: 34.26870
[EarlyStopping] (Update) Best Score: 33.18209
[EarlyStopping] (Update) Best Score: 32.01257
[EarlyStopping] (Update) Best Score: 31.02473
Epoch 00015: reducing learning rate of group 0 to 2.5000e-04.
epoch: 16, loss: 29.4768, val_loss: 50.9621
[EarlyStoppi

In [ ]:
# yogiyo 오버샘플 모델링
NUM_EPOCHS = 20
STATEDICT_OV_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_OV_clean.pt'

best_loss = np.inf

for epoch in tqdm(range(NUM_EPOCHS), total=NUM_EPOCHS):
    loss = train(model, train_loader, optimizer, loss_fn, device)

    val_loss = evaluate(model, test_loader, loss_fn, device)

    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), STATEDICT_OV_PATH)

    if epoch % 5 == 0:
        print(f'epoch: {epoch+1}, loss: {loss:.4f}, val_loss: {val_loss:.4f}')

    # Early Stop
    es(loss)
    if es.early_stop:
        break

    # Scheduler
    scheduler.step(val_loss)

model.load_state_dict(torch.load(STATEDICT_OV_PATH))
torch.save(model.state_dict(), f'/content/drive/MyDrive/project_final/data/models/yogiyo_OV_clean{best_loss:.4f}.pt')

  0%|          | 0/20 [00:00<?, ?it/s]

epoch: 1, loss: 84.5334, val_loss: 63.7501
[EarlyStopping] (Update) Best Score: 84.53337
[EarlyStopping] (Update) Best Score: 47.77351
[EarlyStopping] (Update) Best Score: 27.10025
[EarlyStopping] (Update) Best Score: 19.20366
[EarlyStopping] (Update) Best Score: 14.91649
epoch: 6, loss: 12.2629, val_loss: 16.6054
[EarlyStopping] (Update) Best Score: 12.26286
[EarlyStopping] (Update) Best Score: 10.51378
[EarlyStopping] (Update) Best Score: 9.32866
[EarlyStopping] (Update) Best Score: 7.94201
[EarlyStopping] (Update) Best Score: 7.23426
epoch: 11, loss: 6.3380, val_loss: 15.9132
[EarlyStopping] (Update) Best Score: 6.33805
[EarlyStopping] (Update) Best Score: 5.73936
Epoch 00012: reducing learning rate of group 0 to 5.0000e-04.
[EarlyStopping] (Update) Best Score: 4.21585
[EarlyStopping] (Update) Best Score: 3.38987
[EarlyStopping] (Update) Best Score: 3.13252
Epoch 00015: reducing learning rate of group 0 to 2.5000e-04.
epoch: 16, loss: 2.4544, val_loss: 15.6164
[EarlyStopping] (Updat

# 결과 확인하기

In [ ]:
# # yogiyo 전체데이터 모델링
# model.load_state_dict(torch.load(STATEDICT_PATH))
# random_evaluation(model, test_dataset, dataset.wordvocab.index2word, device)

질문   : 진한 미역국 한 그릇 맛있게 먹었습니다
답변   : 안녕하세요 맛 나게 드셨다니 정말 감사합니다 뚝배기 한 그릇 씩 조리 하고 있어서 재료 양 맛 의 풍 미 식감 등등 을 항상 일정하게 조리 할수
예측답변: 감사합니다 좋은 휴일 되십시요
질문   : 맛 나게 잘먹엇슴돠 감사함 돠
답변   : 저희 목살 맛있게 잘 드셨다니 기쁘고 힘 이 납니다 소중한 리뷰 와 저희 에게 큰 힘 이 되는 좋은 평가 도 남겨 주셔서 정말 감사 드립니다
예측답변: 고객 님 주문 해 주셔서 감사합니다 맛있게 드시고 예쁜 리뷰 남겨주셔서 감사해요 피스타치오 가 갈리는 품목 이 프루타 과일 과 디저트 로 비타민 충전
질문   : 포장 이 너무 예뻐요 진짜 가서 먹는듯 한 플레이 팅 파스타 에 마늘 이 듬뿍 들어있더라고요 역시 마늘 의 민족 미트볼 이랑 가라 아케 도 맛있었어요
답변   : 명작 파스타 입니다 맛있게 드셨다니 저희 도 기분 이 좋습니다 정성스런 리뷰 도 너무 감사 드립니다 앞 으로도 더 맛있는 파스타 제공 드릴 수 있도록 직원
예측답변: 감사합니다 좋은 밤 되세요
질문   : 어렸을 때 부터 시켜 먹던 오래된 맛집 이 예요 맛있고 배달 도 빠르고 좋습니다
답변   : 안녕하세요 좋은 기억 의 만다린 이네요 감사 드립니다 계속 섬심 성의 를 다해 모시겠습니다 건강하세요
예측답변: 안녕하세요 마라하 오죽 전 점 입니다 주문 해주시고 잘 드셔주셔서 정말 감사해요 덕분 에 행복지수 마구 상승 하네요 저희 음식 드시면서 힘 많이 내셨음 좋겠습니다 즐거운
질문   : 분식점 으로 충실한 그 맛 식후감
답변   : 짧지만 강렬한 리뷰 감사 드립니다 깨 돌이 김밥 의 떡볶이 소스 는 캡사이신 이 들어가지 않습니다 고추장 과 청양고추 가루 로 매운 맛 을 내서 추억
예측답변: 감사합니다 좋은 밤 되세요
질문   : 국밥 집 중 에 젤 맛있어요 굿
답변   : 고객 님 소중한 고객 니 이 이임 국밥 집 중 에 젤 맛있다니 지금 까지 본 리뷰 중 에 제일 뿌듯하고 힘 이 나 는 말

In [ ]:
# yogiyo 오버샘플 모델링(앞에 패딩)
model.load_state_dict(torch.load(STATEDICT_OV_PATH))
random_evaluation(model, test_dataset, dataset.wordvocab.index2word, device)

질문   : 고기 가 질겨 요
답변   : 주방 에 전달 토록 하겠습니다 식사 에 불편 드려 죄송합니다
예측답변: 주방 에 전달 토록 하겠습니다 식사 에 불편 드려 죄송합니다
질문   : 눈꽃 빙수 가 양도 많고 딸기 수박 서비스 과일 도 다 싱싱하고 맛있었어요 스테비 아 감자 도 꿀맛 이 요 번창 하세요
답변   : 고객 님 주문 해 주셔서 감사합니다 맛있게 드시고 작품 사진 까지 올려주셔서 고맙습니다 항상 신선하고 달달 한 과일 과 디저트 보내 드리기 위해 꾸준히 신경 쓰겠습니다
예측답변: 고객 님 주문 해 주셔서 감사합니다 맛있게 드시고 작품 사진 까지 올려주셔서 고맙습니다 항상 신선하고 달달 한 과일 과 디저트 보내 드리기 위해 꾸준히 신경 쓰겠습니다
질문   : 배달 착오 가 있었지만 잘 먹었습니다
답변   : 죄송합니다 좋은 밤 되십시요
예측답변: 죄송합니다 좋은 밤 되십시요
질문   : 리뷰 계란찜 을 시켰는데 조리 중 엎어졌고 야채 추가 로 리뷰 대신 한 다 고 하셔서 어이없었지만 넘겼습니다 어찌 되었던 서비스 제품 이니 근데 야채 가
답변   : 불편 을 드려 죄송합니다 리뷰 서비스 지적 사항 은 고객 님 알림 내용 을 리뷰 를 지우고 자유 선택 1 으로 수정 했습니다 옥수수 는 델리
예측답변: 불편 을 드려 죄송합니다 리뷰 서비스 지적 사항 은 고객 님 알림 내용 을 리뷰 를 지우고 자유 선택 1 으로 수정 했습니다 옥수수 는 델리
질문   : 정말 리뷰 잘 안 남기는데 완전 별로 였어요 이돈 주기 에도 먹기 아까운 초밥 이 다 저렴 이 들 로만 구성 되어있고 회도 오래된듯 한 느낌
답변   : 저희 는 가성 비 를 자랑 하는 초밥 입니다 매 일 아침 손질 하는건 기본 이 고요 연어 는 그라브락스 연어 를 사용 하기 때문 에
예측답변: 저희 는 가성 비 를 자랑 하는 초밥 입니다 매 일 아침 손질 하는건 기본 이 고요 연어 는 그라브락스 연어 를 사용 하기 때문 에
질문   : 페퍼로니 피자 를 주문 했습니다 기름기 가득하고 짭짤한 피자 를 

In [ ]:
# yogiyo 오버샘플 모델링(뒤에 패딩)
model.load_state_dict(torch.load(STATEDICT_OV_PATH))
random_evaluation(model, test_dataset, dataset.wordvocab.index2word, device)

질문   : 이 걸 뭐라고 해야 할지 아직도 안 왔네요 배달 예 정시 간 이 30분 인데 65분 지나도 안 옴
답변   : 배송 이 늦어진 부분 정말 죄송합니다 조리 는 빨리 되나 저희 가 이용 하고 있는 배달 대행 업체 의 문제 로 피크타임 배송 이 늦는 경우
예측답변: 배송 이 늦어진 부분 정말 죄송합니다 조리 는 빨리 되나 저희 가 이용 하고 있는 배달 대행 업체 의 문제 로 피크타임 배송 이 늦는 경우
질문   : 음식 의 양은 많지만 맛 이 있지는 않네요 다 못 먹고 남겼습니다 사진 에 있는게 다 남은 음식 이에요 특히 절 인 양파 는 무슨 맛
답변   : 안녕하세요 고객 님 정말 정말 죄송합니다 부족한 부분 으로 실망 시켜 드린 점 사과 드리며 항상 저희 는 리뷰 덕분 에 더욱 책임감 을 갖게 되는데요
예측답변: 안녕하세요 고객 님 정말 정말 죄송합니다 부족한 부분 으로 실망 시켜 드린 점 사과 드리며 항상 저희 는 리뷰 덕분 에 더욱 책임감 을 갖게 되는데요
질문   : 이렇게 왔네요 기분 나빠서 버렸어요
답변   : 왕 빈자 삼파전 입니다 고객 님 믿고 주문 해주셨는데 불편 을 드려서 죄송합니다 항상 기본 적 으로 제일 신경 쓰고 있는 부분 인데 도 불구 하고
예측답변: 왕 빈자 삼파전 입니다 고객 님 믿고 주문 해주셨는데 불편 을 드려서 죄송합니다 항상 기본 적 으로 제일 신경 쓰고 있는 부분 인데 도 불구 하고
질문   : 오 돌뼈 순 한 맛 이라 전혀 맵 지 않았구요 계란찜 추가 했는데 맛있게 잘 먹었어요 배달 빨리 해주셨고 친절하셨습니다
답변   : 소중한 리뷰 감사합니다 순 한 맛 부탁 하셔서 순 한 맛 해드려는데 더 노력 하겠 습니다
예측답변: 양 이 맛 나게 에 맛있게 드셨다니 다행 새해 복 많이 애용 해주세요
질문   : 맛 은 평범합니다 영수증 에 주소 까지나 와있는데 전화하셔서 제대로 안 나와있다고 하셔서 사과 하고 음식 받았습니다 기분 나쁜 식사 만 아니면 더 좋을 뻔했네요
답변   : 기분 나쁘셨다니 죄송합니다 고객 님 이 